In [1]:
from tokenizers import ByteLevelBPETokenizer
import tensorflow as tf
import numpy as np
from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import NFKC, Sequence, Lowercase
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer

In [2]:
tokenizer = Tokenizer(BPE())
tokenizer.normalizer = Sequence([
    Lowercase()
])
tokenizer.pre_tokenizer = ByteLevel()
tokenizer.decoder = ByteLevelDecoder()

In [3]:
trainer = BpeTrainer(vocab_size=50000, inital_alphabet=ByteLevel.alphabet(), special_tokens=[
            "<s>",
            "<pad>",
            "</s>",
            "<unk>",
            "<mask>"
        ])
tokenizer.train(["austen-emma.txt"], trainer)

In [4]:
import os
os.mkdir('tokenizer_gpt')

In [5]:
tokenizer.save("tokenizer_gpt/tokenizer.json")

In [6]:
#pip install transformers

In [7]:
#pip install tf-keras --user 

In [8]:
#pip install --upgrade transformers

In [9]:
from transformers import GPT2TokenizerFast, GPT2Config, TFGPT2LMHeadModel

In [10]:
tokenizer_gpt = GPT2TokenizerFast.from_pretrained("tokenizer_gpt")

In [11]:
tokenizer_gpt.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})

0

In [12]:
tokenizer_gpt.eos_token_id
# 2

2

In [13]:
tokenizer_gpt.encode("<s> this is </s>")
# [0, 265, 157, 56, 2]

[0, 265, 157, 56, 2]

In [14]:
config = GPT2Config(
  vocab_size=tokenizer_gpt.vocab_size,
  bos_token_id=tokenizer_gpt.bos_token_id,
  eos_token_id=tokenizer_gpt.eos_token_id
)
model = TFGPT2LMHeadModel(config)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [15]:
config

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.56.1",
  "use_cache": true,
  "vocab_size": 11750
}

In [16]:
with open("austen-emma.txt", "r", encoding='utf-8') as f:
    content = f.readlines()

In [17]:
content_p = []
for c in content:
    if len(c)>10:
        content_p.append(c.strip())
content_p = " ".join(content_p)+tokenizer_gpt.eos_token

In [18]:
tokenized_content = tokenizer_gpt.encode(content_p)

In [19]:
examples = []
block_size = 100
BATCH_SIZE = 12
BUFFER_SIZE = 1000
for i in range(0, len(tokenized_content)):
    examples.append(tokenized_content[i:i + block_size])

In [20]:
train_data = [] 
labels = [] 
for example in examples: 
    train_data.append(example[:-1]) 
    labels.append(example[1:])

In [21]:
import tensorflow as tf

In [22]:
dataset = tf.data.Dataset.from_tensor_slices((train_data[:1000], labels[:1000]))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [23]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [24]:
num_epoch = 10
history = model.fit(dataset, epochs=num_epoch)

Epoch 1/10

83/83 [==============================] - 656s 7s/step - loss: 6.6495 - accuracy: 0.1093
Epoch 2/10
83/83 [==============================] - 581s 7s/step - loss: 3.5868 - accuracy: 0.3809
Epoch 3/10
83/83 [==============================] - 587s 7s/step - loss: 1.7779 - accuracy: 0.7161
Epoch 4/10
83/83 [==============================] - 568s 7s/step - loss: 0.8135 - accuracy: 0.8868
Epoch 5/10
83/83 [==============================] - 570s 7s/step - loss: 0.4230 - accuracy: 0.9435
Epoch 6/10
83/83 [==============================] - 580s 7s/step - loss: 0.2607 - accuracy: 0.9667
Epoch 7/10
83/83 [==============================] - 579s 7s/step - loss: 0.1774 - accuracy: 0.9786
Epoch 8/10
83/83 [==============================] - 620s 7s/step - loss: 0.1308 - accuracy: 0.9836
Epoch 9/10
83/83 [==============================] - 629s 8s/step - loss: 0.1010 - accuracy: 0.9871
Epoch 10/10
83/83 [==============================] - 621s 7s/step - loss: 0.0812 - accuracy: 0.9896


In [25]:
def generate(start, model):  
    input_token_ids = tokenizer_gpt.encode(start, return_tensors='tf')  
    output = model.generate(  
        input_token_ids,  
        max_length = 500,  
        num_beams = 5,  
        temperature = 0.7,  
        no_repeat_ngram_size=2,  
        num_return_sequences=1  
    )  
    return tokenizer_gpt.decode(output[0])

In [26]:
generate(" ", model)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


"  her mother had died too long ago for her to have more than an indistinct remembrance of her caresses; and her place had been supplied by an excellent woman as governess, who had fallen little short of a mother in affection. sixteen years had miss taylor been in mr. woodhouse's family, less as a governess than a friend, very fond of both daughters, but particularly of emma.  between _them_ it was more the intimacy of any restraint; these were the nominal office of authority being now long passed away, they had ceased to hold the mildness of sisters. she had hardly allowed her friend very mutually attached, the shadow of having been living together as friend and the disadvantages which threatened alloy to impose any disagreeable consciousness.--miss taylor's judgment, and emma doing just what she liked; but directed chiefly by his house from a little too much her own way, however, that they did not at present so unperceived, mournful thought as misfortunes sorrow came--a gentle sorrow

In [27]:
def generate(start, model):  
    input_token_ids = tokenizer_gpt.encode(start, return_tensors='tf')  
    output = model.generate(  
        input_token_ids,  
        max_length = 30,  
        num_beams = 5,  
        temperature = 0.7,  
        no_repeat_ngram_size=2,  
        num_return_sequences=1  
    )  
    return tokenizer_gpt.decode(output[0])
generate("wetson was very good", model)
# "wetson was very good the of her.  the was a, and a of miss taylor; and had, but of a had been a friend

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'wetson was very good, the bride-people gone, her father and herself were left to dine together, with no prospect of a third to'

In [28]:
os.mkdir('my_gpt-2')
model.save_pretrained("my_gpt-2/")

In [29]:
model_reloaded = TFGPT2LMHeadModel.from_pretrained("my_gpt-2/")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at my_gpt-2/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [30]:
tokenizer_gpt.save_pretrained("tokenizer_gpt_auto/")

('tokenizer_gpt_auto/tokenizer_config.json',
 'tokenizer_gpt_auto/special_tokens_map.json',
 'tokenizer_gpt_auto/vocab.json',
 'tokenizer_gpt_auto/merges.txt',
 'tokenizer_gpt_auto/added_tokens.json',
 'tokenizer_gpt_auto/tokenizer.json')